# Анализ данных и процессов
## Лабораторная работа №2. Муха в треугольнике.
### Исходные данные.
Имеется треугольник с вершинами A, B, C. В начальный момент времени муха находится в вершине A. Каждую секунду муха перелетает в другую вершину или остается на месте. Ее поведение задается графом марковского процесса (смотри приложенный рисунок).

![](media/fly.png)

In [227]:
import numpy as np
import pandas as pd

i, j, k = 2, 3, 1

p = np.array([
    [0, j / (j + k), k / (j + k)],
    [i / (i + j), 0, j / (i + j)],
    [i / (i + j + k), j / (i + j + k), k / (i + j + k)]
])

LABELS = ["A", "B", "C"]

pd.DataFrame(p, index=LABELS, columns=LABELS)

,A,B,C
A,0.000000,0.75,0.250000
B,0.400000,0.00,0.600000
C,0.333333,0.50,0.166667


### Задание 1. Необходимо определить среднее время, через которое муха вернётся в вершину А

#### Составление системы уравнений

In [228]:
EQ_LABELS = ['x', 'y', 'z', 'Свободный коэффициент']

a = np.append(
    np.vstack([p.transpose(), [1] * len(p)]),
    np.zeros((len(p) + 1, 1)), axis=1
) - np.eye(len(p) + 1)

pd.DataFrame(a, columns=EQ_LABELS)

,x,y,z,Свободный коэффициент
0,-1.00,0.4,0.333333,0.0
1,0.75,-1.0,0.500000,0.0
2,0.25,0.6,-0.833333,0.0
3,1.00,1.0,1.000000,-1.0


#### Преобразование системы уравнений

In [229]:
def to_upper_triangular(matrix):
    # Копируем матрицу, чтобы не изменять оригинал
    a = matrix.copy()
    rows, cols = a.shape

    for i in range(rows):
        # Находим ведущий элемент
        pivot = a[i, i]
        if pivot == 0:
            continue  # Если ведущий элемент равен нулю, пропускаем итерацию

        for j in range(i + 1, rows):
            # Вычисляем коэффициент для обнуления элемента ниже ведущего
            factor = a[j, i] / pivot
            a[j] = a[j] - factor * a[i]

    return a


a = np.delete(to_upper_triangular(a), len(a) - 2, axis=0)
pd.DataFrame(a, columns=EQ_LABELS)


,x,y,z,Свободный коэффициент
0,-1.0,0.4,0.333333,0.0
1,0.0,-0.7,0.750000,0.0
2,0.0,0.0,2.833333,-1.0


#### Решение системы уравнений

In [230]:
b = -a[:,-1]
a = np.delete(a, -1, axis=1)
solvation = np.linalg.solve(a, b)
for i, label in enumerate(EQ_LABELS[:-1]):
    print(f'{label} = {solvation[i]}')

x = 0.26890756302521013
y = 0.3781512605042017
z = 0.35294117647058826


#### Среднее количество переходов до возвращения

In [231]:
for i, label in enumerate(LABELS):
    print(f'{label} = {1 / solvation[i]}')

A = 3.7187499999999996
B = 2.644444444444444
C = 2.833333333333333


### Задание 2. Написать программу, которая имитирует поведение мухи и выводит среднее количество переходов до первого возвращения в точку А. Сравните результаты.

#### Класс симулятора

In [232]:
class Simulator:
    def __init__(self, move_matrix: np.ndarray):
        self._move_matrix = move_matrix

    def __call__(self, initial_state: int, iters: int) -> list:
        state_counts = [0] * len(self._move_matrix)
        current_state = initial_state
        
        for _ in range(iters):
            state_counts[current_state] += 1
            current_state = self._change_state(current_state)
                
        return state_counts

    def _change_state(self, current_state: int) -> int:
        seed, cumulative_probability = np.random.random(), 0

        for index, probability in enumerate(self._move_matrix[current_state]):
            cumulative_probability += probability
            if seed < cumulative_probability:
                return index
          

#### Имитация поведения мухи

In [233]:
simulator = Simulator(p)
initial_state = 0
N = 10000

counts = simulator(initial_state, N)

print("Число посещений вершин")
for i, label in enumerate(LABELS):
    print(f"{label}: {counts[i]}")

print(f"Среднее количество переходов до возвращения")
for i, label in enumerate(LABELS):
    print(f"{LABELS[i]} = {N / counts[i]}")

Число посещений вершин
A: 2689
B: 3754
C: 3557
Среднее количество переходов до возвращения
A = 3.718854592785422
B = 2.6638252530633992
C = 2.81135788585887


## Вывод
В ходе выполнения лабораторной работы была решена задача "Муха в треугольнике" с помощью эргодических цепей Маркова. Сначала были получены теоретические(ожидаемые) значения среднего количества переходов до возвращения для каждой вершины. Затем были получены эмпирические значения с помощью программы имитирующей поведение мухи. Полученные значения совпали с точностью до 10-х. 